# Open Translate - Full Stack Google Colab Runner

- Repository: https://github.com/simonliu-ai-product/open-translate.git

- This script prepares the environment and launches the server directly from the repository code.

## 關於作者

Simon Liu

Google 人工智慧開發者專家 (AI GDE)

一位人工智慧解決方案領域的技術愛好者，專注於協助企業實施生成式人工智慧 (GAI)、MLOps 和大型語言模型 (LLM) 技術，以推動數位轉型和實際技術應用。

目前也是 Google GenAI 領域的開發者專家 (GDE)，積極參與技術社區，透過技術文章、演講和實踐經驗分享，推動人工智慧技術的應用和發展。迄今為止，已在 Medium 等平台發表了百餘篇技術文章，涵蓋生成式人工智慧、RAG 和 AI 代理等主題，並在許多技術研討會上擔任演講嘉賓，分享人工智慧和生成式人工智慧的實際應用。

相關連結：

- 個人社群媒體連結：https://simonliuyuwei.my.canva.site/link-in-bio

In [1]:
import os
import time
import subprocess
import requests
import threading


# I. Setup requirements

In [2]:
def install_dependencies():
    print("Installing system dependencies (Node.js 20)...")
    !curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
    !sudo apt-get install -y nodejs
    print("Installing Python dependencies...")
    !pip install -q fastapi uvicorn pyngrok nest-asyncio transformers torch accelerate pillow opencc python-multipart python-dotenv sqlalchemy psycopg2-binary pymysql


# II. Main execution

## 1. Get Tokens

In [3]:

try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    ngrok_token = userdata.get('NGROK_TOKEN')
    os.environ["HF_TOKEN"] = hf_token
except Exception:
    print("Error: HF_TOKEN and NGROK_TOKEN must be set in Colab Secrets.")

## 2. Install Dependencies

In [4]:
install_dependencies()

Installing system dependencies (Node.js 20)...
2026-01-16 16:02:51 - Installing pre-requisites
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,297 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,869 kB]
Get:12 https://ppa.launchpadcontent.net

## 3. Clone repository if not exists


In [5]:
if not os.path.exists("open-translate"):
    print("Cloning repository...")
    !git clone https://github.com/simonliu-ai-product/open-translate.git
    %cd open-translate

Cloning repository...
Cloning into 'open-translate'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 46 (delta 11), reused 43 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 1.99 MiB | 24.30 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/open-translate


## 4. Build Frontend


In [6]:
print("Building frontend... (This takes about 1-2 minutes)")
if os.path.exists("frontend"):
    !cd frontend && npm install && npm run build
    !cp -r frontend/dist .
else:
    print("Warning: 'frontend' directory not found.")


Building frontend... (This takes about 1-2 minutes)
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 91 packages, and audited 92 packages in 8s
⠋
⠋17 packages are looking for funding
⠋  run `npm fund` for details
⠋
found 0 vulnerabilities
⠋
> frontend@1.0.0 build
> vite build

Proxying /api to: http://localhost:8000
vite v7.3.1 building client environment for production...
✓ 1750 modules transformed.
dist/index.html                  0.53 kB │ gzip:  0.35 kB
dist/assets/index-_Rze3HMh.js  235.58 kB │ gzip: 77.43 kB
✓ built in 5.28s
⠙

## 5. Start Server in background


In [7]:
# We set PYTHONPATH to include the backend directory for correct imports
env = os.environ.copy()
env["PYTHONPATH"] = os.path.join(os.getcwd(), "backend")

server_process = subprocess.Popen(
    ["python", "backend/main.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=env
)

# Thread to print server logs in real-time
def print_logs(proc):
    for line in proc.stdout:
        print(f"[Server] {line.strip()}")

log_thread = threading.Thread(target=print_logs, args=(server_process,))
log_thread.daemon = True
log_thread.start()

## 6. Wait for server to be healthy


In [8]:
max_retries = 60
ready = False
for i in range(max_retries):
    try:
        # Check the health endpoint
        response = requests.get("http://localhost:8000/api/health", timeout=2)
        if response.status_code == 200 and response.json().get("model_loaded"):
            ready = True
            break
    except:
        pass
    time.sleep(10)
    if i % 3 == 0:
        print(f"Waiting for model to load... ({i*10}s)")

if not ready:
    print("Error: Server timed out during model loading.")
    server_process.terminate()

Waiting for model to load... (0s)
[Server] 2026-01-16 16:03:58.180151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
[Server] WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
[Server] E0000 00:00:1768579438.200683    2420 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
[Server] E0000 00:00:1768579438.206686    2420 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[Server] W0000 00:00:1768579438.222143    2420 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
[Server] W0000 00:00:1768579438.222169    2420 computation_placer.cc:177] computation placer alr

## 7. Finally, setup Ngrok


In [9]:
from pyngrok import ngrok
import nest_asyncio

ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(8000).public_url

print("\n" + "="*60)
print("🎉 MODEL IS READY AND STANDBY!")
print(f"👉 OPEN TRANSLATE UI: {public_url}")
print("="*60 + "\n")


🎉 MODEL IS READY AND STANDBY!
👉 OPEN TRANSLATE UI: https://13f70422ab45.ngrok-free.app

